In [82]:
# retreive and scrape thesis titles from Worldcat

from bs4 import BeautifulSoup
import requests
import time 
import re
thesislist = []

def Worldcat(url):
    global thesislist
    print(url)
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')

    for book in soup.find_all('td', class_='result details'):
        oclc_number = book.find('div', attrs={'oclc_number'}).get_text(strip=True)
        authors = book.find('div', attrs={'author'}).get_text(strip=True)
        author = authors.split(";")[0].split("by ")[1]
        title = book.find('div', attrs={'name'}).get_text(strip=True)
        try:
            publisher = book.find('div', attrs={'publisher'}).get_text(strip=True)
            year = re.search('\d{4}', publisher).group()
        except AttributeError:
            publisher = ""
            year = ""
        d = {"author": author, "title": title, "year": year, "publisher": publisher, "oclc_number": oclc_number, "overig": authors}
        thesislist.append(d)
    
    try:
        important_page = soup.findAll('a', href=True, text='Next')
        time.sleep(2)
        Worldcat('http://www.worldcat.org'+ important_page[0]['href'])
    except:
        print("Finished")

url='https://www.worldcat.org/search?q=pb%3A%28Erasmus+Universiteit%29+AND+su%3A%28scripties%29+AND+kw%3A%28FHKW+OR+Kunstwetenschappen+OR+Maatschappijgeschiedenis+OR+History%29+AND+ot%3A%2815.%3F%29&qt=owc_search'
Worldcat(url)

In [83]:
# import thesis titles in Pandas Dataframe

import pandas as pd

df = pd.DataFrame(thesislist)

In [84]:
# retreive additional theses from thesis.eur.nl

headers = {
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:62.0) Gecko/20100101 Firefox/62.0',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
    'Accept-Language': 'en-US,en;q=0.5',
    'Referer': 'https://thesis.eur.nl/',
    'Content-Type': 'application/x-www-form-urlencoded',
    'Connection': 'keep-alive',
    'Upgrade-Insecure-Requests': '1',
}

data = {
  'format': 'csl',
  'style': '',
  'query': '{"sort":"auto","from":0,"facets":[{"title":"Type","field_id":"type","max_terms":100,"max_display_terms":5,"sort_by":"count","sort_order":"descending","filters":[],"partOf_start":0,"type":"default"},{"title":"Publicly Available","field_id":"public","max_terms":100,"max_display_terms":5,"sort_by":"term","sort_order":"descending","filters":[],"partOf_start":0,"type":"boolean"},{"title":"Year","field_id":"issued_date","max_terms":100,"max_display_terms":10,"sort_by":"term","sort_order":"descending","filters":[],"partOf_start":0,"type":"year"},{"title":"Affiliation","field_id":"affiliation_label_partOf","max_terms":100,"max_display_terms":5,"sort_by":"count","sort_order":"descending","filters":[{"term":"Erasmus School of History, Culture and Communication"}],"partOf_start":0,"type":"partOf"},{"title":"Series","field_id":"host_series_label","max_terms":100,"max_display_terms":5,"sort_by":"count","sort_order":"descending","filters":[{"term":"Maatschappijgeschiedenis / History of Society"},{"term":"RM - Early Modern Intellectual History"}],"partOf_start":0,"type":"default"}],"took":8,"filters":{"values":[{"query":"","field_id":"all"}],"options":[{"field_id":"all","title":"All Fields"},{"field_id":"title","title":"Title"},{"field_id":"author","title":"Author"},{"field_id":"affiliation","title":"Affiliation"},{"field_id":"series","title":"Series"}]},"sortings":[{"field":[{"field_id":"_score","order":"descending"},{"field_id":"issued_date","order":"descending"}],"id":"score","title":"Score / Year"},{"field":[{"field_id":"title_label","order":"ascending"}],"id":"title","title":"Title"},{"field":[{"field_id":"issued_date","order":"descending"}],"id":"year_desc","title":"Year (descending)"},{"field":[{"field_id":"issued_date","order":"ascending"}],"id":"year_asc","title":"Year (ascending)"}],"total":267,"old_total":267}'
}

response = requests.post('https://thesis.eur.nl/search', headers=headers, data=data)
data = response.json()

In [85]:
thesislist2 = []

for i in data:
    url = i['URL']
    title = i['title']
    year = i['issued']['date-parts'][0][0]
    try:
        author = i['author'][0]["name"]
    except:
        author = ""
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')
    try:
        subtitle = soup.find('p', class_="lead subtitle").get_text(strip=True)
        print("subtitle found: " + subtitle)
    except:
        subtitle = ""
    
    title = title + ". " + subtitle
    d = {"author": author, "title": title, "url": url, "year": year, }
    thesislist2.append(d)
    print(year, title)
    time.sleep(2)

In [86]:
# thesis repository titles to pandas dataframe

df2 = pd.DataFrame(thesislist2)
df2.head()

,author,title,url,year
0,"Kavak, A",Nederlands-Indië in de ogen van Nederlandse re...,http://hdl.handle.net/2105/42777,2018
1,"Aarts, M.",Selama-lamanya? Noties van Indisch-heid bij Ro...,http://hdl.handle.net/2105/42779,2017
2,"Loon, I. van",Bemiddelen na een bedrijfssluiting: De integra...,http://hdl.handle.net/2105/42780,2017
3,H.J. Moeliker,Het bekeerde zelf: Een onderzoek naar de genea...,http://hdl.handle.net/2105/41338,2017
4,J.F. Ebeltjes,Mahanisme in Nederland 1890-1914.,http://hdl.handle.net/2105/39324,2017


In [129]:
# merge datasets
new_df = df[['author', 'title', 'year']].copy()
new_df2 = df2[['author', 'title', 'year']].copy()
new_df = new_df.append(new_df2)
new_df.reset_index(inplace=True)
new_df = new_df.drop('index', axis=1)
new_df.head()

,author,title,year
0,Jolien Harmsen,You gave us the dreaming : Aboriginal law and ...,1993
1,Naomi van Stapele,"Kudé? kudedi! try or die : ethnicity, class a...",2007
2,Merel van Ekdom,Finance of heritage factories,2007
3,Estrid Bos,From paganism to Christianity,2005
4,Erik Vellinga,Pride and prejudice : Anglo-Indians and racial...,1994


In [130]:
len(new_df)

923

In [131]:
# count duplicate authors
len(pd.concat(g for _, g in new_df.groupby("title") if len(g) > 1))

52

In [132]:
# drop duplicates by author
new_df.drop_duplicates(['author'], keep='last', inplace=True)

In [133]:
# find missed duplicates by title
pd.concat(g for _, g in new_df.groupby("title") if len(g) > 1)

,author,title,year
854,"Bouter, K. de",Curbing Communism. American motivations for in...,2013
855,"Bouter, Kyle de",Curbing Communism. American motivations for in...,2013
718,"Beek, Julisa van",The Forming of a Community in the Heart of the...,2013
856,"Beek, J.J. van",The Forming of a Community in the Heart of the...,2013
866,Sule E.O.,The Netherlands development cooperation with A...,2013
887,"Sule, E.O.",The Netherlands development cooperation with A...,2011
671,,The United States and the Founding of NATO. Ho...,2017
804,Stef Schuthof,The United States and the Founding of NATO. Ho...,2017


In [134]:
# remove missed duplicates by title 
new_df.drop(854, inplace=True)
new_df.drop(856, inplace=True)
new_df.drop(866, inplace=True)
new_df.drop(671, inplace=True)
len(new_df)

895

In [141]:
new_df.dtypes

author    object
title     object
year      object
dtype: object

In [142]:
new_df['year'] = new_df['year'].astype(int)
new_df.head()

,author,title,year
0,Jolien Harmsen,You gave us the dreaming : Aboriginal law and ...,1993
1,Naomi van Stapele,"Kudé? kudedi! try or die : ethnicity, class a...",2007
2,Merel van Ekdom,Finance of heritage factories,2007
3,Estrid Bos,From paganism to Christianity,2005
4,Erik Vellinga,Pride and prejudice : Anglo-Indians and racial...,1994


In [148]:
new_df = new_df.sort_values('year')
new_df.reset_index(inplace=True)
new_df = new_df.drop('index', axis=1)
new_df.head()

,author,title,year
0,Jean Spanjersberg,De institutionele groei van het Nederlandse on...,1983
1,Dick de Winter,Disciplinering van protestgedrag van havenarbe...,1983
2,Christi Boerdam,Ongehuwde moeders te Rotterdam in het laatste ...,1983
3,B W H Guyt,De visserij op de Noordzee : de lage landen aa...,1983
4,Harma Tent,Het beeld van de Amerikaanse indiaan in Iberis...,1983


In [150]:
new_df.to_csv("thesis_titles.csv", encoding='utf-8')